In [ ]:
####### Innovation Project #######
### Ecole Centrale Paris - Ecole CentraleSupélec###
### Laboratoire MSSMat ###
### Advisor : Pierre JEHEL ###
### Students : Felipe DA SILVA BRANDAO, Thibaud FRITZ, Louis-Henri MICHALIK ###
### Date : 2015-2016 ###

## MultivariateStochasticProcesses ##
#Edit 1 : 12/01/2015


from pylab import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import dblquad
from scipy.integrate import quad   
from random import *
import time



def R_(b1,b2,X,Y):
    r2=(X/b1)**2+(Y/b2)**2
    return r2*np.exp(-r2)

def FT1(b,X):
#Fourier transform of exp(-x**2/b**2)
    return b/(np.sqrt(2))*np.exp(-(b*X/2)**2)

def f2(b,X):
    return X**2/b**2*exp(-X**2/b**2)

def FT2(b,X):
#Fourier Transform of x**2/b**2*exp(-x**2/b**2)
    return -b/(4*np.sqrt(2))*(b**2*X**2-2)*np.exp(-b**2*X**2/4)

def S_(b1,b2,X,Y):
#2D Fourier Transform of R_
    return 1/(4*np.pi)*(FT2(b1,X)*FT1(b2,Y)+FT2(b2,Y)*FT1(b1,X))

def Sgauss_(b1,b2,X,Y):
    return b1*b2/(4*np.pi)*np.exp(-(b1*X/2)**2-(b2*Y/2)**2)

def integ(b1,b2,a,b):
    return dblquad(lambda X,Y: S_(b1,b2,X,Y),-a,a,lambda X: -b,lambda X: b)[0]
    
    
def plotR(b1,b2):
    fig = figure()
    ax = Axes3D(fig)
    X = np.arange(0, 5*b1, 0.1)
    Y = np.arange(0, 5*b2, 0.1)
    X, Y = np.meshgrid(X, Y)
    R = R_(b1,b2,X,Y)

    ax.plot_surface(X, Y, R, rstride=1, cstride=1, cmap=cm.coolwarm,linewidth=0, antialiased=False)
    #ax.contourf(X, Y, R, zdir='z', offset=-2, cmap=cm.hot)
    ax.set_zlim(0,1.5)
    plt.show()
    
    
#1 : Solaire, 2 : Eolien, 3 : Compost
def R11(X,Y):
    b1=2
    b2=2
    return np.exp(-(X/b1)**2-(Y/b2)**2)

def R22(X,Y):
    b1=3
    b2=3
    return np.exp(-(X/b1)**2-(Y/b2)**2)

def R33(X,Y):
    b1=5
    b2=5
    return np.exp(-(X/b1)**2-(Y/b2)**2)

def R12(X,Y):
    b1=5
    b2=5
    r2=(X/b1)**2+(Y/b2)**2
    return r2*np.exp(-r2)

def R13(X,Y):
    b1=3
    b2=3
    r2=(X/b1)**2+(Y/b2)**2
    return r2*np.exp(-r2)

def R23(X,Y):
    b1=2
    b2=2
    r2=(X/b1)**2+(Y/b2)**2
    return r2*np.exp(-r2)

def S11(X,Y):
    b1=2
    b2=2
    return Sgauss_(b1,b2,X,Y)

def S22(X,Y):
    b1=3
    b2=3
    return Sgauss_(b1,b2,X,Y)

def S33(X,Y):
    b1=4
    b2=4
    return Sgauss_(b1,b2,X,Y)

def S12(X,Y):
    b1=8
    b2=8
    return -Sgauss_(b1,b2,X,Y)

def S13(X,Y):
    b1=3
    b2=3
    return -Sgauss_(b1,b2,X,Y)

def S23(X,Y):
    b1=2
    b2=2
    return -Sgauss_(b1,b2,X,Y)

def plotS(b1,b2):
    fig = figure()
    ax = Axes3D(fig)
    X = np.arange(0, 5*b1, 0.1)
    Y = np.arange(0, 5*b2, 0.1)
    X, Y = np.meshgrid(X, Y)
    S = S11(X,Y)

    ax.plot_surface(X, Y, S, rstride=1, cstride=1, cmap=cm.coolwarm,linewidth=0, antialiased=False)
    #ax.contourf(X, Y, R, zdir='z', offset=-2, cmap=cm.hot)
    ax.set_zlim(0,1.5)
    plt.show()

def cholesky(A):
    """Performs a Cholesky decomposition of A, which must 
    be a symmetric and positive definite matrix. The function
    returns the lower variant triangular matrix, L."""
    n = len(A)

    # Create zero matrix for L
    L = [[0.0] * n for i in range(n)]

    # Perform the Cholesky decomposition
    for i in range(n):
        for k in range(i+1):
            tmp_sum = sum(L[i][j] * L[k][j] for j in range(k))
            
            if (i == k): # Diagonal elements
                if A[i][i] - tmp_sum>=0:
                    L[i][k] = sqrt(A[i][i] - tmp_sum)
                else:
                    L[i][k] = 1j*sqrt(-A[i][i] + tmp_sum)
            else:
                L[i][k] = (1.0 / L[k][k] * (A[i][k] - tmp_sum))
    return L

def PowerSpectral(X,Y):
    S=zeros((3,3))
    S[0,0]=S11(X,Y)
    S[0,1]=S12(X,Y)
    S[0,2]=S13(X,Y)
    S[1,0]=S12(X,Y)
    S[1,1]=S22(X,Y)
    S[1,2]=S23(X,Y)
    S[2,0]=S13(X,Y)
    S[2,1]=S23(X,Y)
    S[2,2]=S33(X,Y)
    return S

def H(X,Y):
    S=PowerSpectral(X,Y)
    return cholesky(S)



ku=1
N=32
N1=32
N2=32
Deltak=ku/N
Deltak1=ku/N1
Deltak2=ku/N2
L1=3*2*np.pi/Deltak1 #=150
L2=3*2*np.pi/Deltak2
M1=256
M2=256
DeltaX=2*np.pi/(Deltak1*M1) 
DeltaY=2*np.pi/(Deltak2*M2)

def Theta(X,Y):
    Theta=zeros((3,3))
    for i in range(3):
        for j in range(3):
            if i>=j:
                if real(H(X,Y)[i][j])!=0:
                    Theta[i][j]=math.atan(imag(H(X,Y)[i][j])/real(H(X,Y)[i][j]))
                else:
                    Theta[i][j]=sign(imag(H(X,Y)[i][j]))*np.pi/2

    return Theta

def Phi():
    Phi=zeros((3,2,M1,M2))
    for i in range(3):
        for j in range(2):
            for k in range(M1):
                for l in range(M2):
                    Phi[i][j][k][l]=2*np.pi*rand()
    return Phi


def K1():
    K1=zeros((3,M1))
    for i in range(3):
        for j in range(M1):
            K1[i][j]=j*Deltak1+(i+1)/3*Deltak1
    return K1
    
def K2():
    K2=zeros((3,M2))
    for i in range(3):
        for j in range(M2):
            K2[i][j]=j*Deltak2+(i+1)/3*Deltak2
    return K2


def MatB(M1_,M2_):
    K1_=K1()
    K2_=K2()
    Phi_=Phi()
    MatB=np.zeros((3,3,2,M1_,M2_))
    for i in range(3):
        for j in range(3):
            for k in range(2):
                for l in range(M1_):
                    for m in range(M2_):
                        if j>i: #car Hrs est triangulaire inférieure
                            MatB[i][j][k][l][m]=0
                        else:
                            if k==0: #k=0 -> I2=-1 // k=1 -> I2=1
                                temp1=H(K1_[j][l],-K2_[j][m])
                                temp2=Theta(K1_[j][l],-K2_[j][m])
                                MatB[i][j][k][l][m]=2*abs(temp1[i][j])*np.sqrt(Deltak1*Deltak2)*np.exp(-1j*temp2[i][j])*np.exp(1j*Phi_[j][k][l][m])
                            else:
                                temp1=H(K1_[j][l],K2_[j][m])
                                temp2=Theta(K1_[j][l],K2_[j][m])
                                MatB[i][j][k][l][m]=2*abs(temp1[i][j])*np.sqrt(Deltak1*Deltak2)*np.exp(-1j*temp2[i][j])*np.exp(1j*Phi_[j][k][l][m])
    return MatB

def matrix_function_g(M1_,M2_):
    B=MatB(M1_,M2_)
    g=np.zeros((3,3,M1_,M2_))
    for r in range(3):
        for s in range(3):
            g[r,s,:,:]=M1_*M2_*np.fft.ifft2(B[r,s,1,:,:])+np.fft.fft2(B[r,s,0,:,:])
    return g

def matrix_function_f1(M1_,M2_):
    g=matrix_function_g(M1_,M2_)
    f1=np.zeros((3*M1_,3*M2_))
    for i in range(3*M1_):
        for j in range(3*M2_):
            p=i%M1_
            q=j%M2_                
            f1[i,j]=real(g[0,0,p,q]*np.exp(1j*(i*DeltaX*Deltak1/3+j*DeltaY*Deltak2/3)))
    return f1
    
def matrix_function_f2(M1_,M2_):
    g=matrix_function_g(M1_,M2_)
    f2=np.zeros((3*M1_,3*M2_))
    for i in range(3*M1_):
        for j in range(3*M2_):
            p=i%M1_
            q=j%M2_
            f2[i,j]=real(g[1,0,p,q]*np.exp(1j*(i*DeltaX*Deltak1/3+j*DeltaY*Deltak2/3))+g[1,1,p,q]*np.exp(1j*(i*DeltaX*2*Deltak1/3+j*DeltaY*2*Deltak2/3)))
    return f2


def matrix_function_f3(M1_,M2_):
    g=matrix_function_g(M1_,M2_)
    f3=np.zeros((3*M1_,3*M2_))
    for i in range(3*M1_):
        for j in range(3*M2_):
            p=i%M1_
            q=j%M2_
            f3[i,j]=real(g[2,0,p,q]*np.exp(1j*(i*DeltaX*Deltak1/3+j*DeltaY*Deltak2/3))+g[2,1,p,q]*np.exp(1j*(i*DeltaX*2*Deltak1/3+j*DeltaY*2*Deltak2/3))+g[2,2,p,q]*np.exp(1j*(i*DeltaX*Deltak1+j*DeltaY*Deltak2)))
    return f3

def plotf1():
    
    fig = plt.figure()
    ax=Axes3D(fig)
    X = np.arange(0, L1, DeltaX)
    Y = np.arange(0, L2, DeltaY)
    X,Y=np.meshgrid(X,Y)

    f1=matrix_function_f1(M1,M2)
    #ax.plot_surface(X, Y, f1, rstride=8, cstride=5, alpha=0.55)

    ax.set_xlabel('X')
    ax.set_xlim(0, L1)
    ax.set_ylabel('Y')
    ax.set_ylim(0, L2)
    ax.set_zlabel('Z')
    ax.set_zlim(-4,4)

    #plt.show()

    cset = ax.contour(X, Y, f1, zdir='z', offset=0, cmap=cm.coolwarm)
   
    plt.show()  


def plotf2():
    
    fig = plt.figure()
    ax=Axes3D(fig)
    X = np.arange(0, L1, DeltaX)
    Y = np.arange(0, L2, DeltaY)
    X,Y=np.meshgrid(X,Y)

    f2=matrix_function_f2(M1,M2)
    #ax.plot_surface(X, Y, f1, rstride=8, cstride=5, alpha=0.55)

    ax.set_xlabel('X')
    ax.set_xlim(0, L1)
    ax.set_ylabel('Y')
    ax.set_ylim(0, L2)
    ax.set_zlabel('Z')
    ax.set_zlim(-4,4)

    #plt.show()

    cset = ax.contour(X, Y, f2, zdir='z', offset=0, cmap=cm.coolwarm)
   
    plt.show()  
    
def plotf3():
    
    fig = plt.figure()
    ax=Axes3D(fig)
    X = np.arange(0, L1, DeltaX)
    Y = np.arange(0, L2, DeltaY)
    X,Y=np.meshgrid(X,Y)

    f3=matrix_function_f3(M1,M2)
    ax.plot_surface(X, Y, f1, rstride=8, cstride=5, alpha=0.55)

    ax.set_xlabel('X')
    ax.set_xlim(0, L1)
    ax.set_ylabel('Y')
    ax.set_ylim(0, L2)
    ax.set_zlabel('Z')
    ax.set_zlim(-4,4)

    plt.show()

    #cset = ax.contour(X, Y, f3, zdir='z', offset=0, cmap=cm.coolwarm)
   
    #plt.show()  
    
def contourplot():
    
    Deltak=ku/N
    Deltak1=ku/N1
    Deltak2=ku/N2
    L1=3*2*np.pi*N1/30 
    L2=3*2*np.pi*N2/30
    L=min(L1,L2)
    M1=256
    M2=256
    DeltaX=2*np.pi/(Deltak1*M1) 
    DeltaY=2*np.pi/(Deltak2*M2)
    
    levels=np.array([-3,-2,-1,-0.5,-0.25,0,0.25,0.5,1,2,3])
     
    fig1 = plt.figure()
    X = np.arange(0, L1, DeltaX)
    Y = np.arange(0, L2, DeltaY)
    
    M1_=len(X) #3*M1/10
    M2_=len(Y)
    M_=min(M1_,M2_)
    
    def max_(H):
        M=H[0,0]
        m=H[0,0]
        for i in range(len(H)):
            M=max(M,max(H[i]))
            m=min(m,min(H[i]))
        return [m,M]    
    
    f1=matrix_function_f1(M1_,M2_)
    H1=np.zeros((M1_,M2_))
    for i in range(M1_):
        for j in range(M2_):
            H1[i][j]=f1[i][j]
            
    [m1,M1]=max_(H1)
    
    levels1=np.array([m1,0.66*m1,0.33*m1,0.15*m1,0.05*m1,0,0.05*M1,0.15*M1,0.33*M1,0.66*M1,M1])
    CS1=plt.contourf(X, Y, H1, zdir='z', offset=0,levels=levels1, alpha=1)
    cbar = plt.colorbar(CS1)
    
    fig2 = plt.figure()
    X = np.arange(0, L1, DeltaX)
    Y = np.arange(0, L2, DeltaY)
    
    f2=matrix_function_f2(M1_,M2_)
    H2=np.zeros((M1_,M2_))
    for i in range(M1_):
        for j in range(M2_):
            H2[i][j]=f2[i][j]
    [m2,M2]=max_(H2)
    
    levels2=np.array([m2,0.66*m2,0.33*m2,0.15*m2,0.05*m2,0,0.05*M2,0.15*M2,0.33*M2,0.66*M2,M2])
    CS2=plt.contourf(X, Y, H2, zdir='z', offset=0,levels=levels2, alpha=1)
    cbar = plt.colorbar(CS2)
    
    fig3 = plt.figure()
    X = np.arange(0, L1, DeltaX)
    Y = np.arange(0, L2, DeltaY)
    
    f3=matrix_function_f3(M1_,M2_)
    H3=np.zeros((M1_,M2_))
    for i in range(M1_):
        for j in range(M2_):
            H3[i][j]=f3[i][j]
    [m3,M3]=max_(H3)
    
    levels3=np.array([m3,0.66*m3,0.33*m3,0.15*m3,0.05*m3,0,0.05*M3,0.15*M3,0.33*M3,0.66*M3,M3])
    CS3=plt.contourf(X, Y, H3, zdir='z', offset=0,levels=levels3, alpha=1)
    cbar = plt.colorbar(CS3)
    
    plt.show() 


In [ ]:
import pylab as pylab
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from random import *
from scipy.integrate import dblquad


def R_(b1,b2,sigma,X,Y):
    return sigma**2*np.exp(-(X/b1)**2-(Y/b2)**2)

def S_(b1,b2,sigma,X,Y):
    return b1*b2*sigma**2/(4*np.pi)*np.exp(-(b1*X/2)**2-(b2*Y/2)**2)

def defA(b1,b2,sigma,N1,N2,M1,M2):
    k1=5./b1
    k2=5./b2
    Deltak1=k1/(N1)
    Deltak2=k2/(N2)
    L1=2*np.pi/(Deltak1)
    L2=2*np.pi/(Deltak2)
    DeltaX=2*np.pi/(M1*Deltak1)
    DeltaY=2*np.pi/(M2*Deltak2)
    X = np.arange(0, L1, DeltaX)
    Y = np.arange(0, L2, DeltaY)
    X, Y = np.meshgrid(X, Y)
    
    K1=[]
    for i in range(0,M1):
        K1+=[i*Deltak1]        
    K2=[]
    for i in range(0,M2):
        K2+=[i*Deltak2]    

    A=np.zeros((M1,M2)).tolist()
    for i in range(1,N1):
        for j in range(1,N2):
            A[i][j]=np.sqrt(2*S_(b1,b2,sigma,K1[i],K2[j])*Deltak1*Deltak2)
    return A

def defA_(b1,b2,sigma,N1,N2,M1,M2):
    k1=5./b1
    k2=5./b2
    Deltak1=k1/(N1)
    Deltak2=k2/(N2)
    L1=2*np.pi/(Deltak1)
    L2=2*np.pi/(Deltak2)
    DeltaX=2*np.pi/(M1*Deltak1)
    DeltaY=2*np.pi/(M2*Deltak2)
    X = np.arange(0, L1, DeltaX)
    Y = np.arange(0, L2, DeltaY)
    X, Y = np.meshgrid(X, Y)
    
    K1=[]
    for i in range(0,M1):
        K1+=[i*Deltak1]        
    K2=[]
    for i in range(0,M2):
        K2+=[i*Deltak2]    

    A_=np.zeros((M1,M2)).tolist()
    for i in range(1,N1):
        for j in range(1,N2):
            A_[i][j]=np.sqrt(2*S_(b1,b2,sigma,K1[i],-K2[j])*Deltak1*Deltak2)
    return A_
            
def defB(b1,b2,sigma,N1,N2,M1,M2):
    Phi1=np.zeros((M1,M2)).tolist()
    for i in range(N1):
        for j in range(N2):
            Phi1[i][j]=2*np.pi*random()

    A=defA(b1,b2,sigma,N1,N2,M1,M2)
    B=np.zeros((M1,M2)).tolist()
    for i in range(N1):
        for j in range(N2):
            B[i][j]=np.sqrt(2)*A[i][j]*np.exp(1j*Phi1[i][j])

    return B

def defB_(b1,b2,sigma,N1,N2,M1,M2):
    Phi2=np.zeros((M1,M2)).tolist()
    for i in range(N1):
        for j in range(N2):
            Phi2[i][j]=2*np.pi*random()
            
    A_=defA_(b1,b2,sigma,N1,N2,M1,M2)
    B_=np.zeros((M1,M2)).tolist()
    for i in range(N1):
        for j in range(N2):
            B_[i][j]=np.sqrt(2)*A_[i][j]*np.exp(1j*Phi2[i][j])

    return B_

def gaussianfield(b1,b2,sigma,N1,N2,M1,M2):   
            
    B=defB(b1,b2,sigma,N1,N2,M1,M2)
    B_=defB_(b1,b2,sigma,N1,N2,M1,M2)

    C=[0 for k in range(M1)]   
    for i in range(M1):
        C[i]=np.fft.ifft(B[i])
        
    D=np.zeros((M2,M1)).tolist()
    for i in range(M2):
        for j in range(M1):
            D[i][j]=C[j][i]
            
    E=[0 for k in range(M2)]
    for j in range(M2):
        E[j]=M1*M2*np.fft.ifft(D[j])
        E[j]=E[j].tolist()
    E1=np.asarray(E)

    F=E1.transpose()


    C_=[0 for k in range(M1)]   
    for i in range(M1):
        C_[i]=np.fft.fft(B_[i])
        
    D_=np.zeros((M2,M1)).tolist()
    for i in range(M2):
        for j in range(M1):
            D_[i][j]=C_[j][i]
            
    E_=[0 for k in range(M2)]
    for j in range(M2):
        E_[j]=M1*np.fft.ifft(D_[j])
        E_[j]=E_[j].tolist()
    E1_=np.asarray(E_)
    
    F_=E1_.transpose()
    
    
    
    H=np.zeros((M2,M1)).tolist()
    for i in range(M2):
        for j in range(M1):
            H[i][j]=np.real(E[i][j]).tolist()+np.real(E_[i][j]).tolist()  
    
    return H
          
    
def theoriticalmaxgaussianfield(b1,b2,sigma,N1,N2,M1,M2):
    C=defA(b1,b2,sigma,N1,N2,M1,M2)
    D=defA_(b1,b2,sigma,N1,N2,M1,M2)
    bound=0
    for i in range(N1):
        for j in range(N2):
            bound+=C[i][j]+D[i][j]
    return np.sqrt(2)*bound

def maximini(H):
    m=H[0][0]
    M=H[0][0]
    M1=len(H[0])
    M2=len(H)
    for i in range(M2):
        for j in range(M1):
            if M<H[i][j]:
                M=H[i][j]
    for i in range(M2):
        for j in range(M1):
            if m>H[i][j]:
                m=H[i][j]
            
    return [m,M]

def probability_that_f_smaller_than_x(b1,b2,sigma,N1,N2,M1,M2,x):
    n=0
    H=gaussianfield(b1,b2,sigma,N1,N2,M1,M2)
    for i in range(M2):
        for j in range(M1):
            if H[i][j]<x:
                n+=1
    return n/(M1*M2)

def probability_that_f_between_x_and_y(b1,b2,sigma,N1,N2,M1,M2,x,y):
    return probability_that_f_smaller_than_x(b1,b2,sigma,N1,N2,M1,M2,y)-probability_that_f_smaller_than_x(b1,b2,sigma,N1,N2,M1,M2,x)

def histogram(b1,b2,sigma,N1,N2,M1,M2):
    fig = plt.figure()
    ax = fig.add_subplot(111)
    maxbounds=theoriticalmaxgaussianfield(b1,b2,sigma,N1,N2,M1,M2)
    H=100*(double(gaussianfield(b1,b2,sigma,N1,N2,M1,M2))+maxbounds)/(2*maxbounds)
    x=[]
    for i in range(M2):
        x+=H[i].tolist()
   
    numBins = 50
    ax.hist(x,numBins,color='green',alpha=0.8)

    plt.show()

def plot(b1,b2,sigma,N1,N2,M1,M2):
    
    maxbounds=maxgaussianfield(b1,b2,sigma,N1,N2,M1,M2)[1]
    zvals = 100*(double(gaussianfield(b1,b2,sigma,N1,N2,M1,M2))+maxbounds)/(2*maxbounds)
   
    cmap = mpl.colors.ListedColormap(['red','yellow','lawngreen','cyan','blue'])
    
    bounds1=maxbounds/20
    bounds2=maxbounds/40
    bounds=[0,45,48,52,55,100]
    norm = mpl.colors.BoundaryNorm(bounds, cmap.N)

    img = plt.imshow(zvals,interpolation='nearest',
                    cmap = cmap,norm=norm)

    plt.colorbar(img,cmap=cmap,
                norm=norm,boundaries=bounds,ticks=[0,45,50,55,100])

    plt.show()
    
def contourplot(b1,b2,sigma,N1,N2,M1,M2):
    k1=5./b1
    k2=5./b2
    Deltak1=k1/(N1)
    Deltak2=k2/(N2)
    #L1=2*np.pi/(Deltak1)
    #L2=2*np.pi/(Deltak2)
    
    DeltaX=2*np.pi/(M1*Deltak1)
    DeltaY=2*np.pi/(M2*Deltak2)
    L1=2*np.pi*N1/5
    L2=2*np.pi*N2/5
    L=min(L1,L2)
    #Correspond à b1=b2=1

    
    fig = plt.figure()
    X = np.arange(0, L1, DeltaX)
    Y = np.arange(0, L2, DeltaY)
    
    H=gaussianfield(b1,b2,sigma,N1,N2,M1,M2)
    a=maximini(H)
    m=a[0]
    M=a[1]
    
    M1_=len(X)
    M2_=len(Y)
    M_=min(M1_,M2_)
    
#    Moyenne_de_I=0.5
#    I=np.zeros((M2_,M1_))
#    for i in range(M2_):
#        for j in range(M1_):
#            if (Moyenne_de_I/0.5)*(H[i][j]-m)/(-m+M)>1:
#                I[i][j]=1
#            else:
#                I[i][j]=(Moyenne_de_I/0.5)*(H[i][j]-m)/(-m+M)
    I=np.zeros((M2_,M1_))
    for i in range(M2_):
        for j in range(M1_):
            I[i][j]=H[i][j]
            #-0.02*(-m+M)


    levels=np.array([m,0.5*m,0.25*m,0.1*m,0,0.1*M,0.25*M,0.5*M,M])
#    levels=np.array([m,0.1*m,0,0.1*M,M])
    
    CS=plt.contourf(X, Y, I, zdir='z', offset=0,levels=levels, alpha=1)
    cbar = plt.colorbar(CS)
    
    fig2 = plt.figure()
    ax = fig2.add_subplot(111)
    x=[]
    for i in range(M2_):
        x+=I[i].tolist()
    numBins = 50
    ax.hist(x,numBins,color='green',alpha=0.8)
    
    
    plt.show()   
    
    def probability_that_H_larger_than_x(I,threshold):
        n=0
        for i in range(M2_):
            for j in range(M1_):
                if I[i][j]>threshold:
                    n+=1
        return n/(M1_*M2_)
    
    #a=probability_that_H_larger_than_x(I,0)
    #return a

def mean(H):
    M=0
    for i in range(len(H)):
        for j in range(len(H[0])):
            M+=H[i][j]
    mean=M/(len(H)*len(H[0]))
    return mean

def variance(H):
    m=mean(H)
    H_=np.zeros((len(H),len(H[0]))).tolist()
    for i in range(len(H)):
        for j in range(len(H[0])):
            H_[i][j]=(H[i][j]-m)**2           
    return mean(H_) 


In [ ]:
####### Innovation Project #######
### Ecole Centrale Paris - Ecole CentraleSupélec###
### Laboratoire MSSMat ###
### Advisor : Pierre JEHEL ###
### Students : Felipe DA SILVA BRANDAO, Thibaud FRITZ, Louis-Henri MICHALIK ###
### Date : 2015-2016 ###

## MultivariateStochasticProcesses ##
#Edit 1 : 12/01/2015


from pylab import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.integrate import dblquad
from scipy.integrate import quad   
from random import *
from scipy import *
from scipy.stats import norm
import time



def R_(b1,b2,X,Y):
    r2=(X/b1)**2+(Y/b2)**2
    return r2*np.exp(-r2)

def FT1(b,X):
#Fourier transform of exp(-x**2/b**2)
    return b/(np.sqrt(2))*np.exp(-(b*X/2)**2)

def f2(b,X):
    return X**2/b**2*exp(-X**2/b**2)

def FT2(b,X):
#Fourier Transform of x**2/b**2*exp(-x**2/b**2)
    return -b/(4*np.sqrt(2))*(b**2*X**2-2)*np.exp(-b**2*X**2/4)

def S_(b1,b2,X,Y):
#2D Fourier Transform of R_
    return 1/(4*np.pi)*(FT2(b1,X)*FT1(b2,Y)+FT2(b2,Y)*FT1(b1,X))

def Sgauss_(b1,b2,X,Y):
    return b1*b2/(4*np.pi)*np.exp(-(b1*X/2)**2-(b2*Y/2)**2)

def integ(b1,b2,a,b):
    return dblquad(lambda X,Y: S_(b1,b2,X,Y),-a,a,lambda X: -b,lambda X: b)[0]
    
    
def plotR(b1,b2):
    fig = figure()
    ax = Axes3D(fig)
    X = np.arange(0, 5*b1, 0.1)
    Y = np.arange(0, 5*b2, 0.1)
    X, Y = np.meshgrid(X, Y)
    R = R_(b1,b2,X,Y)

    ax.plot_surface(X, Y, R, rstride=1, cstride=1, cmap=cm.coolwarm,linewidth=0, antialiased=False)
    #ax.contourf(X, Y, R, zdir='z', offset=-2, cmap=cm.hot)
    ax.set_zlim(0,1.5)
    plt.show()
    
    
#1 : Solaire, 2 : Eolien, 3 : Compost
def R11(X,Y):
    b1=2
    b2=2
    return np.exp(-(X/b1)**2-(Y/b2)**2)

def R22(X,Y):
    b1=3
    b2=3
    return np.exp(-(X/b1)**2-(Y/b2)**2)

def R33(X,Y):
    b1=5
    b2=5
    return np.exp(-(X/b1)**2-(Y/b2)**2)

def R12(X,Y):
    b1=5
    b2=5
    r2=(X/b1)**2+(Y/b2)**2
    return r2*np.exp(-r2)

def R13(X,Y):
    b1=3
    b2=3
    r2=(X/b1)**2+(Y/b2)**2
    return r2*np.exp(-r2)

def R23(X,Y):
    b1=2
    b2=2
    r2=(X/b1)**2+(Y/b2)**2
    return r2*np.exp(-r2)

def S11(X,Y):
    b1=3
    b2=3
    return Sgauss_(b1,b2,X,Y)

def S22(X,Y):
    b1=3
    b2=3
    return Sgauss_(b1,b2,X,Y)

def S33(X,Y):
    b1=3
    b2=3
    return Sgauss_(b1,b2,X,Y)

def S12(X,Y):
    b1=3
    b2=3
    return -0.1*Sgauss_(b1,b2,X,Y)

def S13(X,Y):
    b1=3
    b2=3
    return -0.5*Sgauss_(b1,b2,X,Y)

def S23(X,Y):
    b1=3
    b2=3
    return -0.8*Sgauss_(b1,b2,X,Y)

def Matrix33():
    #return [[0.8,-0.8,-0.1],[-0.8,1,0.1],[-0.1,0.1,5]]
    return [[1,-0.8,-0.1],[-0.8,1,0.1],[-0.1,0.1,1]]
    
def plotS(b1,b2):
    fig = figure()
    ax = Axes3D(fig)
    X = np.arange(0, 5*b1, 0.1)
    Y = np.arange(0, 5*b2, 0.1)
    X, Y = np.meshgrid(X, Y)
    S = S11(X,Y)

    ax.plot_surface(X, Y, S, rstride=1, cstride=1, cmap=cm.coolwarm,linewidth=0, antialiased=False)
    #ax.contourf(X, Y, R, zdir='z', offset=-2, cmap=cm.hot)
    ax.set_zlim(0,1.5)
    plt.show()

def cholesky(A):
    """Performs a Cholesky decomposition of A, which must 
    be a symmetric and positive definite matrix. The function
    returns the lower variant triangular matrix, L."""
    n = len(A)

    # Create zero matrix for L
    L = [[0.0] * n for i in range(n)]

    # Perform the Cholesky decomposition
    for i in range(n):
        for k in range(i+1):
            tmp_sum = sum(L[i][j] * L[k][j] for j in range(k))
            
            if (i == k): # Diagonal elements
                if A[i][i] - tmp_sum>=0:
                    L[i][k] = sqrt(A[i][i] - tmp_sum)
                else:
                    L[i][k] = 1j*sqrt(-A[i][i] + tmp_sum)
            else:
                L[i][k] = (1.0 / L[k][k] * (A[i][k] - tmp_sum))
    return L

def PowerSpectral(X,Y):
    S=zeros((3,3))
    S[0,0]=S11(X,Y)
    S[0,1]=S12(X,Y)
    S[0,2]=S13(X,Y)
    S[1,0]=S12(X,Y)
    S[1,1]=S22(X,Y)
    S[1,2]=S23(X,Y)
    S[2,0]=S13(X,Y)
    S[2,1]=S23(X,Y)
    S[2,2]=S33(X,Y)
    return S

def H(X,Y):
    S=PowerSpectral(X,Y)
    return cholesky(S)



ku=1
N=16
N1=16
N2=16
Deltak=ku/N
Deltak1=ku/N1
Deltak2=ku/N2
L1=3*2*np.pi/Deltak1 #=150
L2=3*2*np.pi/Deltak2
M1=64
M2=64
DeltaX=2*np.pi/(Deltak1*M1) 
DeltaY=2*np.pi/(Deltak2*M2)

def Theta(X,Y):
    Theta=zeros((3,3))
    for i in range(3):
        for j in range(3):
            if i>=j:
                if real(H(X,Y)[i][j])!=0:
                    Theta[i][j]=math.atan(imag(H(X,Y)[i][j])/real(H(X,Y)[i][j]))
                else:
                    Theta[i][j]=sign(imag(H(X,Y)[i][j]))*np.pi/2

    return Theta

def Phi():
    Phi=zeros((3,2,M1,M2))
    for i in range(3):
        for j in range(2):
            for k in range(M1):
                for l in range(M2):
                    Phi[i][j][k][l]=2*np.pi*rand()
    return Phi


def K1():
    K1=zeros((3,M1))
    for i in range(3):
        for j in range(M1):
            K1[i][j]=j*Deltak1+(i+1)/3*Deltak1
    return K1
    
def K2():
    K2=zeros((3,M2))
    for i in range(3):
        for j in range(M2):
            K2[i][j]=j*Deltak2+(i+1)/3*Deltak2
    return K2


def MatBrs():
    K1_=K1()
    K2_=K2()
    Phi_=Phi()
    h=cholesky(Matrix33())
    MatBrs=np.zeros((3,3,2,M1,M2))
    for i in range(3):
        for j in range(3):
            for k in range(2):
                for l in range(M1):
                    for m in range(M2):
                        if j>i:
                            MatBrs[i][j][k][l][m]=0
                        else:
                            MatBrs[i][j][k][l][m]=2*h[i][j]*np.sqrt(Sgauss_(3,3,K1_[j][l],K2_[j][m])*Deltak1*Deltak2)*np.exp(1j*Phi_[j][k][l][m])
    return MatBrs

def MatBs():
    K1_=K1()
    K2_=K2()
    Phi_=Phi()
    MatBs=np.zeros((3,2,M1,M2))
    for j in range(3):
        for k in range(2):
            for l in range(M1):
                for m in range(M2):
                    MatBs[j][k][l][m]=2*np.sqrt(Sgauss_(3,3,K1_[j][l],K2_[j][m])*Deltak1*Deltak2)*np.exp(1j*Phi_[j][k][l][m])
    return MatBs

def matrix_function_g():
    B=MatBs()
    g=np.zeros((3,M1,M2))
    for s in range(3):
        g[s,:,:]=M1*M2*np.fft.ifft2(B[s,1,:,:])+np.fft.fft2(B[s,0,:,:])
    return g

def matrix_function_f1():
    g=matrix_function_g()
    f1=np.zeros((3*M1,3*M2))
    for i in range(3*M1):
        for j in range(3*M2):
            p=i%M1
            q=j%M2                
            f1[i,j]=real(g[0,p,q]*np.exp(1j*(i*DeltaX*Deltak1/3+j*DeltaY*Deltak2/3)))
    return f1
    
def matrix_function_f2():
    g=matrix_function_g()
    f2=np.zeros((3*M1,3*M2))
    for i in range(3*M1):
        for j in range(3*M2):
            p=i%M1
            q=j%M2
            f2[i,j]=real(g[1,p,q]*np.exp(1j*(i*2*DeltaX*Deltak1/3+j*2*DeltaY*Deltak2/3)))
    return f2


def matrix_function_f3():
    g=matrix_function_g()
    f3=np.zeros((3*M1,3*M2))
    for i in range(3*M1):
        for j in range(3*M2):
            p=i%M1
            q=j%M2
            f3[i,j]=real(g[2,p,q]*np.exp(1j*(i*3*DeltaX*Deltak1/3+j*3*DeltaY*Deltak2/3)))
    return f3

def simulated_f1():
    h=cholesky(Matrix33())
    f1=matrix_function_f1()
    simuf1=np.zeros((3*M1,3*M2))
    for i in range(3*M1):
        for j in range(3*M2):
            simuf1[i][j]=h[0][0]*f1[i][j]
    return simuf1

def simulated_f2():
    h=cholesky(Matrix33())
    f1=matrix_function_f1()
    f2=matrix_function_f2()
    simuf2=np.zeros((3*M1,3*M2))
    for i in range(3*M1):
        for j in range(3*M2):
            simuf2[i][j]=h[1][0]*f1[i][j]+h[1][1]*f2[i][j]
    return simuf2

def simulated_f3():
    h=cholesky(Matrix33())
    f1=matrix_function_f1()
    f2=matrix_function_f2()
    f3=matrix_function_f3()
    simuf3=np.zeros((3*M1,3*M2))
    for i in range(3*M1):
        for j in range(3*M2):
            simuf3[i][j]=h[2][0]*f1[i][j]+h[2][1]*f2[i][j]+h[2][2]*f3[i][j]
    return simuf3

def plotf1():
    
    fig = plt.figure()
    ax=Axes3D(fig)
    X = np.arange(0, L1, DeltaX)
    Y = np.arange(0, L2, DeltaY)
    X,Y=np.meshgrid(X,Y)

    f1=simulated_f1()
    ax.plot_surface(X, Y, f1, rstride=8, cstride=5, alpha=0.55)

    ax.set_xlabel('X')
    ax.set_xlim(0, L1)
    ax.set_ylabel('Y')
    ax.set_ylim(0, L2)
    ax.set_zlabel('Z')
    ax.set_zlim(-4,4)

    #plt.show()

    #cset = ax.contour(X, Y, f1, zdir='z', offset=0, cmap=cm.coolwarm)
   
    plt.show()  


def plotf2():
    
    fig = plt.figure()
    ax=Axes3D(fig)
    X = np.arange(0, L1, DeltaX)
    Y = np.arange(0, L2, DeltaY)
    X,Y=np.meshgrid(X,Y)

    f2=simulated_f2()
    ax.plot_surface(X, Y, f2, rstride=8, cstride=5, alpha=0.55)

    ax.set_xlabel('X')
    ax.set_xlim(0, L1)
    ax.set_ylabel('Y')
    ax.set_ylim(0, L2)
    ax.set_zlabel('Z')
    ax.set_zlim(-4,4)

    plt.show()

    #cset = ax.contour(X, Y, f2, zdir='z', offset=0, cmap=cm.coolwarm)
   
    #plt.show()  
    
def plotf3():
    
    fig = plt.figure()
    ax=Axes3D(fig)
    X = np.arange(0, L1, DeltaX)
    Y = np.arange(0, L2, DeltaY)
    X,Y=np.meshgrid(X,Y)

    f3=simulated_f3()
    ax.plot_surface(X, Y, f3, rstride=8, cstride=5, alpha=0.55)

    ax.set_xlabel('X')
    ax.set_xlim(0, L1)
    ax.set_ylabel('Y')
    ax.set_ylim(0, L2)
    ax.set_zlabel('Z')
    ax.set_zlim(-4,4)

    plt.show()

    #cset = ax.contour(X, Y, f3, zdir='z', offset=0, cmap=cm.coolwarm)
   
    #plt.show()  
    
def contourplot():
    

    Deltak=ku/N
    Deltak1=ku/N1
    Deltak2=ku/N2
    L1=3*2*np.pi*N1/60
    L2=3*2*np.pi*N2/60
    L=min(L1,L2)
    M1=64
    M2=64
    DeltaX=2*np.pi/(Deltak1*M1) 
    DeltaY=2*np.pi/(Deltak2*M2)
    
    levels=np.array([-3,-2,-1,-0.5,-0.25,0,0.25,0.5,1,2,3])
     
    fig1 = plt.figure()
    X = np.arange(0, L1, DeltaX)
    Y = np.arange(0, L2, DeltaY)
    
    M1_=len(X) #3*M1/10
    M2_=len(Y)
    M_=min(M1_,M2_)
        
    def max_(H):
        M=H[0,0]
        m=H[0,0]
        for i in range(len(H)):
            M=max(M,max(H[i]))
            m=min(m,min(H[i]))
        return [m,M]    
    
    
    f1=simulated_f1()
    H1=np.zeros((M1_,M2_))
    for i in range(M1_):
        for j in range(M2_):
            H1[i][j]=f1[i][j]
            
    [m1,M1]=max_(H1)
    
    levels1=np.array([m1,0.66*m1,0.33*m1,0.15*m1,0.05*m1,0,0.05*M1,0.15*M1,0.33*M1,0.66*M1,M1])
    CS1=plt.contourf(X, Y, H1, zdir='z', offset=0,levels=levels1, alpha=1)
    cbar = plt.colorbar(CS1)
    
    fig2 = plt.figure()
    X = np.arange(0, L1, DeltaX)
    Y = np.arange(0, L2, DeltaY)
    
    f2=simulated_f2()
    H2=np.zeros((M1_,M2_))
    for i in range(M1_):
        for j in range(M2_):
            H2[i][j]=f2[i][j]
    [m2,M2]=max_(H2)
    
    levels2=np.array([m2,0.66*m2,0.33*m2,0.15*m2,0.05*m2,0,0.05*M2,0.15*M2,0.33*M2,0.66*M2,M2])
    CS2=plt.contourf(X, Y, H2, zdir='z', offset=0,levels=levels2, alpha=1)
    cbar = plt.colorbar(CS2)
    
    fig3 = plt.figure()
    X = np.arange(0, L1, DeltaX)
    Y = np.arange(0, L2, DeltaY)
    
    f3=simulated_f3()
    H3=np.zeros((M1_,M2_))
    for i in range(M1_):
        for j in range(M2_):
            H3[i][j]=f3[i][j]
    [m3,M3]=max_(H3)
    
    levels3=np.array([m3,0.66*m3,0.33*m3,0.15*m3,0.05*m3,0,0.05*M3,0.15*M3,0.33*M3,0.66*M3,M3])
    CS3=plt.contourf(X, Y, H3, zdir='z', offset=0,levels=levels3, alpha=1)
    cbar = plt.colorbar(CS3)
    
    plt.show() 
